In [ ]:
import torch
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

In [ ]:
#pip install transformers

In [ ]:
import spacy
from spacy.tokens import Doc , Span
import re

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
def read_dataset(file_path):
    examples = []
    with open(file_path, 'r', encoding='utf-8') as file:
        example = []
        for line in file:
            line = line.strip()
            if line.startswith('-'):
                if example:
                    examples.append(' '.join(example))
                example = [line]
            else:
                example.append(line)
        if example:
            examples.append(' '.join(example))
    return examples

In [ ]:
def extract_entities(text):
    entity_regex = r'\[(.*?)\]\((.*?)\)'
    entities = re.findall(entity_regex, text)
    return entities

In [ ]:
def preprocess_data(data):
    training_data = []
    for line in data:
        entities = extract_entities(line)
        doc_text = line
        ent_list = []
        for entity, tag in entities:
            start = line.index("[" + entity + "]")
            end = start + len(entity)
            ent_list.append((start, end, tag))

        training_data.append((doc_text, {"entities": ent_list}))
    return training_data

In [ ]:
file_path = 'labeled.txt'
examples = read_dataset(file_path)
training_data = preprocess_data(examples)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
input_texts = [text for text, _ in training_data]

output_texts = [f"{text} [<ENT_START>] {tag} [<ENT_END>]" for text, tags in training_data for tag in tags]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
input_encodings = tokenizer(input_texts, return_tensors="tf", padding=True, truncation=True)
output_encodings = tokenizer(output_texts, return_tensors="tf", padding=True, truncation=True)

In [ ]:
max_sequence_length = max(input_encodings["input_ids"].shape[1], output_encodings["input_ids"].shape[1])
input_encodings = tokenizer(input_texts, return_tensors="tf", padding="max_length", truncation=True, max_length=max_sequence_length)
output_encodings = tokenizer(output_texts, return_tensors="tf", padding="max_length", truncation=True, max_length=max_sequence_length)

In [ ]:
output_ids = output_encodings["input_ids"]
labels = output_ids.numpy().copy()

In [ ]:
#output_texts

In [ ]:
#labels[labels == tokenizer.pad_token_id] = -100

In [ ]:
train_size = int(0.8 * len(input_encodings["input_ids"]))
val_size = int(0.1 * len(input_encodings["input_ids"]))
test_size = int(0.1 * len(input_encodings["input_ids"]))

In [ ]:
train_inputs = {k: v[:train_size] for k, v in input_encodings.items()}
train_labels = output_ids[:train_size]

val_inputs = {k: v[train_size:train_size + val_size] for k, v in input_encodings.items()}
val_labels = output_ids[train_size:train_size + val_size]

test_inputs = {k: v[-test_size:] for k, v in input_encodings.items()}
test_labels = output_ids[-test_size:]

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [ ]:
with strategy.scope():
  model = TFGPT2LMHeadModel.from_pretrained("gpt2")
  model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
num_epochs = 20
batch_size = 64
steps_per_epoch = len(train_inputs["input_ids"]) // batch_size



In [ ]:
model.fit(
    train_inputs, train_labels,
    batch_size=batch_size,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=(val_inputs, val_labels)
)

Epoch 1/20
14/14 [==============================] - 55s 590ms/step - loss: 3.0798 - accuracy: 0.5750 - val_loss: 2.6207 - val_accuracy: 0.6874
Epoch 2/20
14/14 [==============================] - 3s 245ms/step - loss: 1.2581 - accuracy: 0.8451 - val_loss: 1.0382 - val_accuracy: 0.8770
Epoch 3/20
14/14 [==============================] - 3s 242ms/step - loss: 0.6731 - accuracy: 0.8994 - val_loss: 0.5584 - val_accuracy: 0.9068
Epoch 4/20
14/14 [==============================] - 3s 227ms/step - loss: 0.4336 - accuracy: 0.9100 - val_loss: 0.3625 - val_accuracy: 0.9176
Epoch 5/20
14/14 [==============================] - 3s 221ms/step - loss: 0.3328 - accuracy: 0.9164 - val_loss: 0.3325 - val_accuracy: 0.9180
Epoch 6/20
14/14 [==============================] - 4s 271ms/step - loss: 0.2893 - accuracy: 0.9206 - val_loss: 0.2662 - val_accuracy: 0.9209
Epoch 7/20
14/14 [==============================] - 3s 241ms/step - loss: 0.2613 - accuracy: 0.9226 - val_loss: 0.2251 - val_accuracy: 0.9265
Epoch

In [ ]:
test_loss, test_accuracy = model.evaluate(
    test_inputs,
    test_labels,
    batch_size=batch_size
)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

2/2 [==============================] - 1s 71ms/step - loss: 0.1690 - accuracy: 0.9318
Test Loss: 0.16901470720767975
Test Accuracy: 0.9317948222160339


In [ ]:
model.save("C:\\Users\\admin\\Desktop")

In [ ]:
#input_text = " - 1000 manatlıq Albalı plüs kart almaq istəyirəm."

In [ ]:
#input_encoding = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True)

In [ ]:
#predictions = model.predict(input_encoding)
